In [ ]:
import pickle as pkl
import pandas as pd
import numpy as np
import uuid
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Callable, Type
from sklearn.cluster import KMeans
from annoy import AnnoyIndex
import os
from time import time
from tqdm import tqdm
from hnswlib import Index

In [ ]:
with open("out/embeddings.pkl", "rb") as f:
    data = pkl.load(f)
text_df, embeddings_df = data
print(len(text_df), len(embeddings_df))

In [ ]:
class HyperBin:
    def __init__(self, num_hyperplanes: int, num_dims: int) -> None:
        self.uuid = str(uuid.uuid4())
        self.hyperplanes = np.random.normal(size=(num_hyperplanes, num_dims))
        norms = np.linalg.norm(self.hyperplanes, axis=1, keepdims=True)
        self.hyperplanes = self.hyperplanes / norms

    def get_bin(self, point: np.array) -> int:
        dot_products = np.dot(self.hyperplanes, point)
        bin_index = np.where(dot_products > 0, 1, 0)
        bin_index = int("".join(bin_index.astype(str)), 2)
        return bin_index



In [ ]:
embeddings_arr = np.array([e for e in embeddings_df["embeddings"]])
print(embeddings_arr.shape)

def test_bins(num_hyperplanes: int, num_dims: int) -> None:
    h = HyperBin(num_hyperplanes, num_dims)
    bins = []
    for i in range(len(embeddings_arr)):
        point = embeddings_arr[i]
        bins.append(h.get_bin(point))
    plt.hist(bins, bins=2**num_hyperplanes)
    plt.show()
    
test_bins(8, 384)

In [ ]:
def cosine_distance(vector1: np.array, vector2: np.array) -> float:
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    cosine_similarity = dot_product / (norm_vector1 * norm_vector2)
    cosine_distance = 1 - cosine_similarity
    return cosine_distance

def euclidean_distance(vector1: np.array, vector2: np.array) -> float:
    return np.linalg.norm(vector1 - vector2)

class LSHIndex:
    def __init__(self, num_hyperplanes: int, num_dims: int, num_bins: int, distance_func: Callable[[np.array, np.array], float], **kwargs) -> None:
        self._all_points = []
        self.dinstance_func = distance_func
        self._hyperbins = [HyperBin(num_hyperplanes, num_dims) for _ in range(num_bins)]
        self._index: dict[str, dict[int, list[np.array]]] = {bin.uuid: {} for bin in self._hyperbins}

    def index(self, point: np.array) -> None:
        self._all_points.append(point)
        for bin in self._hyperbins:
            bin_index = bin.get_bin(point)
            if bin_index not in self._index[bin.uuid]:
                self._index[bin.uuid][bin_index] = []
            self._index[bin.uuid][bin_index].append(point)

    def search(self, point: np.array, use_index: bool, limit: int) -> list[tuple[float, str]]:
        candidates = []
        if use_index:
            c_hashes = set()
            for bin in self._hyperbins:
                bin_index = bin.get_bin(point)
                if bin_index in self._index[bin.uuid]:
                    for candidate in self._index[bin.uuid][bin_index]:
                        c_hash = hash(candidate.tobytes())
                        if c_hash not in c_hashes:
                            candidates.append(candidate)
                            c_hashes.add(c_hash)
        else:
            candidates = self._all_points

        distances = []
        for candidate in candidates:
            distance = self.dinstance_func(candidate, point)
            distances.append((distance, candidate))
            
        distances.sort(key=lambda x: x[0])
        return [{"distance": d, "point_hash": hash(a.tobytes())} for d, a in distances[:limit]]


In [ ]:
index = LSHIndex(8, 384, 1, distance_func=cosine_distance)
for embedding in embeddings_arr:
    index.index(embedding)

bin_sizes = []
for uid, bin in index._index.items():
    for bin_id, points in bin.items():
        bin_sizes.append(len(points))

print(f"Average bin size: {np.mean(bin_sizes)}")
print(f"Max bin size: {np.max(bin_sizes)}")
print(f"Min bin size: {np.min(bin_sizes)}")
index.search(embeddings_arr[0], use_index=True, limit=10)

In [ ]:
def benchmark(
        embeddings: list[np.array],
        distance_fun: Callable[[np.array, np.array], float],
        index_cls: Type[LSHIndex],
        search_limit: int,
        test_sample_size: int,
) -> None:
    def count_misses(num_of_bins: int, limit: int, num_of_hyperplanes: int, distance_fun) -> int:
        index = index_cls(num_hyperplanes=num_of_hyperplanes, num_dims=384, num_bins=num_of_bins, distance_func=distance_fun, embeddings=embeddings)
        for embedding in embeddings[:-test_sample_size]:
            index.index(embedding)
        misses = 0
        for random_embedding in embeddings[-test_sample_size:]:
            with_index = index.search(random_embedding, use_index=True, limit=limit)
            no_index = index.search(random_embedding, use_index=False, limit=limit)
            diff = len(set([x["point_hash"] for x in with_index]) - set([x["point_hash"] for x in no_index]))
            misses += diff
        return misses

    data = []
    for num_of_bins in range(1, 11):
        for num_of_hyperplanes in range(4, 10):
            misses = count_misses(num_of_bins, limit=search_limit, num_of_hyperplanes=num_of_hyperplanes, distance_fun=cosine_distance)
            data.append((num_of_bins, num_of_hyperplanes, misses))
            print(f"num_of_bins: {num_of_bins}, num_of_hyperplanes: {num_of_hyperplanes}, missed: {misses}/{test_sample_size * search_limit}")

    df = pd.DataFrame(data, columns=["num_of_bins", "num_of_hyperplanes", "misses"])
    df = df.pivot(index="num_of_bins", columns="num_of_hyperplanes", values="misses")

    title=f"{index_cls.__name__} {distance_fun.__name__} misses"
    ax = plt.axes()
    sns.heatmap(df, annot=True, fmt="d", ax=ax)
    ax.set_title(title)

In [ ]:
embeddings_arr_sample = np.array([np.array(e) for e in embeddings_arr[:10_000]])
print(embeddings_arr_sample.shape)
test_sample_size = 100
search_limit = 30
distance_fun = cosine_distance

In [ ]:
benchmark(
    embeddings=embeddings_arr_sample,
    distance_fun=cosine_distance,
    index_cls=LSHIndex,
    search_limit=search_limit,
    test_sample_size=test_sample_size,
)

In [ ]:
class CenterdHyperBin(HyperBin):
    def __init__(self, num_hyperplanes: int, center: np.array) -> None:
        self.uuid = str(uuid.uuid4())
        num_dims = len(center)
        self.hyperplanes = np.random.normal(size=(num_hyperplanes, num_dims))
        self.hyperplanes = self.hyperplanes - center  # shift hyperplanes to cross on the center of mass

        norms = np.linalg.norm(self.hyperplanes, axis=1, keepdims=True)
        self.hyperplanes = self.hyperplanes / norms

class CenteredLSHIndex(LSHIndex):
    def __init__(self, num_hyperplanes: int, num_bins: int, distance_func: Callable[[np.array, np.array], float], embeddings: list[np.array], **kwargs) -> None:
        self._all_points = []
        self.dinstance_func = distance_func
        center = np.mean(embeddings, axis=0)
        self._hyperbins = [CenterdHyperBin(num_hyperplanes, center) for _ in range(num_bins)]
        self._index: dict[str, dict[int, list[np.array]]] = {bin.uuid: {} for bin in self._hyperbins}
        self._hash_cache = {}

        for point in self._all_points:
            self.index(point)

benchmark(
    embeddings=embeddings_arr_sample,
    distance_fun=cosine_distance,
    index_cls=CenteredLSHIndex,
    search_limit=search_limit,
    test_sample_size=test_sample_size,
)

In [ ]:
class KMeansHyperBin(HyperBin):
    def __init__(self, hyperplanes: np.array) -> None:
        self.uuid = str(uuid.uuid4())
        self.hyperplanes = hyperplanes
        norms = np.linalg.norm(self.hyperplanes, axis=1, keepdims=True)
        self.hyperplanes = self.hyperplanes / norms

class KMeansLSHIndex(LSHIndex):
    def __init__(self, num_hyperplanes: int, num_bins: int, distance_func: Callable[[np.array, np.array], float], embeddings: list[np.array], **kwargs) -> None:
        self._all_points = []
        self.dinstance_func = distance_func

        # Use kmeans to initialize hyperplanes
        kmeans = KMeans(n_clusters=num_hyperplanes, n_init="auto").fit(embeddings)
        hyperplanes = kmeans.cluster_centers_

        self._hyperbins = [KMeansHyperBin(hyperplanes) for _ in range(num_bins)]
        self._index: dict[str, dict[int, list[np.array]]] = {bin.uuid: {} for bin in self._hyperbins}
        self._hash_cache = {}

        for point in self._all_points:
            self.index(point)

benchmark(
    embeddings=embeddings_arr_sample,
    distance_fun=cosine_distance,
    index_cls=KMeansLSHIndex,
    search_limit=search_limit,
    test_sample_size=test_sample_size,
)

In [ ]:
data = []
for i in range(4, 15):
    number_of_hyperplanes = i
    start_time = time()
    kmeans_index = KMeansLSHIndex(num_hyperplanes=9, num_bins=1, distance_func=cosine_distance, embeddings=embeddings_arr)
    end_time = time()
    init_time = end_time - start_time
    print(f"number_of_hyperplanes: {number_of_hyperplanes}, init_time: {init_time}")
    data.append((number_of_hyperplanes, init_time))




In [ ]:
df = pd.DataFrame(data, columns=["number_of_hyperplanes", "init_time_s"])
ax = plt.axes()

sns.pointplot(data=df, x="number_of_hyperplanes", y="init_time_s", ax=ax, linestyles="--")
ax.set_title("KMeansLSHIndex init time s by number of hyperplanes")
plt.show()

In [ ]:
annoy_metrics = ["angular", "euclidean", "manhattan", "hamming", "dot"]

def annoy_count_misses(
    embeddings: list[np.array],
    distance_fun: Callable[[np.array, np.array], float],
    annoy_metric: str,
    search_limit: int,
    test_sample_size: int,
    num_trees: int,
) -> int:

    dimension = embeddings[0].shape[0]
    index = AnnoyIndex(dimension, metric=annoy_metric)
    
    # Indexing
    for i, embedding in enumerate(embeddings[:-test_sample_size]):
        index.add_item(i, embedding)
    index.build(num_trees)

    misses = 0
    for i, random_embedding in enumerate(embeddings[-test_sample_size:]):
        with_index = index.get_nns_by_vector(random_embedding, search_limit)
        
        # Compute distances to all points and get the closest ones
        distances = [(j, distance_fun(random_embedding, embeddings[j])) for j in range(len(embeddings))]
        distances.sort(key=lambda x: x[1])
        no_index = [j for j, _ in distances[:search_limit]]

        diff = len(set(with_index) - set(no_index))
        misses += diff

    return misses

def benchmark_annoy(
    embeddings: list[np.array],
    distance_fun: Callable[[np.array, np.array], float],
    search_limit: int,
    test_sample_size: int,
) -> None:
    
    data = []
    for num_trees in range(10, 100, 10):  
        for metric in annoy_metrics:
            misses = annoy_count_misses(embeddings, distance_fun, metric, search_limit, test_sample_size, num_trees)
            data.append((num_trees, misses, metric))
            print(f"num_trees: {num_trees}, metric: {metric}, missed: {misses}/{test_sample_size * search_limit}")

    df = pd.DataFrame(data, columns=["num_trees", "misses", "metric"])
    df = df.pivot(index="num_trees", columns="metric", values="misses")
    
    title = f"Annoy vs {distance_fun.__name__} misses"
    ax = plt.axes()
    sns.heatmap(df, annot=True, fmt="d", ax=ax)
    ax.set_title(title)

benchmark_annoy(
    embeddings=embeddings_arr_sample,
    distance_fun=cosine_distance,
    search_limit=search_limit,
    test_sample_size=test_sample_size,
)

In [ ]:
def benchmark_annoy_index(embeddings: list[np.array], trees: int) -> dict[str, float]:
    os.makedirs("out", exist_ok=True)
    file_name = "out/index.ann"

    start = time()
    index = AnnoyIndex(384, metric="angular")
    for i, embedding in enumerate(embeddings):
        index.add_item(i, embedding)
    index.build(trees)
    indexing_time = time() - start

    start = time()
    index.save(file_name)
    end = time()
    index_save_time = end - start

    start = time()
    index = AnnoyIndex(384, metric="angular")
    index.load(file_name)
    end = time()
    index_load_time = end - start

    return {
        'indexing_time': indexing_time,
        'save_time': index_save_time,
        'load_time': index_load_time,
        'index_size': os.path.getsize(file_name) / 1024 / 1024
    }

def benchmark_annoy_index_and_plot(embeddings_ar, embeddings_step, trees_range, trees_step) -> None:
    results = pd.DataFrame(columns=['trees', 'embeddings', 'indexing_time', 'save_time', 'load_time', 'index_size'])

    for i in tqdm(range(embeddings_step, len(embeddings_ar), embeddings_step)):
        embeddings = embeddings_ar[:i] 
        for trees in range(trees_step, trees_range, trees_step):
            stats = benchmark_annoy_index(embeddings, trees)
            data = {
                'trees': trees,
                'embeddings': len(embeddings),
                'indexing_time': stats['indexing_time'],
                'save_time': stats['save_time'],
                'load_time': stats['load_time'],
                'index_size': stats['index_size']
            }
            print(data)
            results = pd.concat([results, pd.DataFrame(data, index=[0])])
    
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))
    indexing_time = results.pivot('trees', 'embeddings', 'indexing_time')
    save_time = results.pivot('trees', 'embeddings', 'save_time')
    load_time = results.pivot('trees', 'embeddings', 'load_time')
    index_size = results.pivot('trees', 'embeddings', 'index_size')

    sns.heatmap(indexing_time, ax=axs[0, 0], cmap='YlGnBu', annot=True, fmt=".5f")
    axs[0, 0].set_title('Indexing Time s')

    sns.heatmap(save_time, ax=axs[0, 1], cmap='YlGnBu', annot=True, fmt=".5f")
    axs[0, 1].set_title('Save Time s')

    sns.heatmap(load_time, ax=axs[1, 0], cmap='YlGnBu', annot=True, fmt=".5f")
    axs[1, 0].set_title('Load Time s')

    sns.heatmap(index_size, ax=axs[1, 1], cmap='YlGnBu', annot=True, fmt=".2f")
    axs[1, 1].set_title('Index Size MB')

    plt.tight_layout()
    plt.show()

benchmark_annoy_index_and_plot(embeddings_arr, 25_000, 60, 10)


In [ ]:
hnsw_metrics = ["l2", "ip", "cosine"]

def hnsw_count_misses(
    embeddings: list[np.array],
    distance_fun: Callable[[np.array, np.array], float],
    hnsw_metric: str,
    search_limit: int,
    test_sample_size: int,
    num_links: int,
) -> int:

    dimension = embeddings[0].shape[0]
    index = Index(space=hnsw_metric, dim=dimension)
    
    # Indexing
    index.init_index(max_elements=len(embeddings), ef_construction=200, M=num_links)
    index.add_items(np.array(embeddings[:-test_sample_size]))

    misses = 0
    for i, random_embedding in enumerate(embeddings[-test_sample_size:]):
        with_index = index.knn_query(random_embedding, k=search_limit)[0][0]

        # Compute distances to all points and get the closest ones
        distances = [(j, distance_fun(random_embedding, embeddings[j])) for j in range(len(embeddings))]
        distances.sort(key=lambda x: x[1])
        no_index = [j for j, _ in distances[:search_limit]]

        diff = len(set(with_index) - set(no_index))
        misses += diff

    return misses

def benchmark_hnsw(
    embeddings: list[np.array],
    distance_fun: Callable[[np.array, np.array], float],
    search_limit: int,
    test_sample_size: int,
) -> None:
    
    data = []
    for num_links in range(10, 100, 10):  
        for metric in hnsw_metrics:
            misses = hnsw_count_misses(embeddings, distance_fun, metric, search_limit, test_sample_size, num_links)
            data.append((num_links, misses, metric))
            print(f"num_links: {num_links}, metric: {metric}, missed: {misses}/{test_sample_size * search_limit}")

    df = pd.DataFrame(data, columns=["num_links", "misses", "metric"])
    df = df.pivot(index="num_links", columns="metric", values="misses")
    
    title = f"HNSW vs {distance_fun.__name__} misses"
    ax = plt.axes()
    sns.heatmap(df, annot=True, fmt="d", ax=ax)
    ax.set_title(title)

benchmark_hnsw(
    embeddings=embeddings_arr_sample,
    distance_fun=cosine_distance,
    search_limit=search_limit,
    test_sample_size=test_sample_size,
)

In [ ]:

def benchmark_hnsw_index(embeddings: list[np.array], links: int) -> dict[str, float]:
    os.makedirs("out", exist_ok=True)
    file_name = "out/index.hnsw"

    start = time()
    index = Index(space="l2", dim=embeddings[0].shape[0])
    index.init_index(max_elements=len(embeddings), ef_construction=200, M=links)
    index.add_items(embeddings)
    indexing_time = time() - start

    start = time()
    index.save_index(file_name)
    end = time()
    index_save_time = end - start

    start = time()
    index = Index(space="l2", dim=embeddings[0].shape[0])
    index.load_index(file_name)
    end = time()
    index_load_time = end - start

    return {
        'indexing_time': indexing_time,
        'save_time': index_save_time,
        'load_time': index_load_time,
        'index_size': os.path.getsize(file_name) / 1024 / 1024
    }

def benchmark_hnsw_index_and_plot(embeddings_ar, embeddings_step, links_range, links_step) -> None:
    results = pd.DataFrame(columns=['links', 'embeddings', 'indexing_time', 'save_time', 'load_time', 'index_size'])

    for i in tqdm(range(embeddings_step, len(embeddings_ar), embeddings_step)):
        embeddings = embeddings_ar[:i] 
        for links in range(links_step, links_range, links_step):
            stats = benchmark_hnsw_index(embeddings, links)
            data = {
                'links': links,
                'embeddings': len(embeddings),
                'indexing_time': stats['indexing_time'],
                'save_time': stats['save_time'],
                'load_time': stats['load_time'],
                'index_size': stats['index_size']
            }
            print(data)
            results = pd.concat([results, pd.DataFrame(data, index=[0])])
    
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))
    indexing_time = results.pivot('links', 'embeddings', 'indexing_time')
    save_time = results.pivot('links', 'embeddings', 'save_time')
    load_time = results.pivot('links', 'embeddings', 'load_time')
    index_size = results.pivot('links', 'embeddings', 'index_size')

    sns.heatmap(indexing_time, ax=axs[0, 0], cmap='YlGnBu', annot=True, fmt=".5f")
    axs[0, 0].set_title('Indexing Time s')

    sns.heatmap(save_time, ax=axs[0, 1], cmap='YlGnBu', annot=True, fmt=".5f")
    axs[0, 1].set_title('Save Time s')

    sns.heatmap(load_time, ax=axs[1, 0], cmap='YlGnBu', annot=True, fmt=".5f")
    axs[1, 0].set_title('Load Time s')

    sns.heatmap(index_size, ax=axs[1, 1], cmap='YlGnBu', annot=True, fmt=".2f")
    axs[1, 1].set_title('Index Size MB')

    plt.tight_layout()
    plt.show()
benchmark_hnsw_index_and_plot(embeddings_arr, 25_000, 60, 10)